# Problema 1

You may produce seven products by consuming three materials. The unit sales price and material consumption of each product are listed in Table 1. For each day, the supply of these three materials are limited. The supply limits are listed in Table 2. For each day, you need to determine the production quantity for each product.

#### Importando bibliotecas

In [2]:
import pandas as pd
import pyomo.environ as pyo
import os
from cyipopt import *

#### Instanciando dados o problema

In [172]:
df_p1t1 = pd.read_excel('06_quiz_data.xlsx', sheet_name='p1_t1')
df_p1t1.columns = df_p1t1.columns.str.lower()
df_p1t1

,produtos,preco,material_1,material_2,material_3
0,1,100,0,3,10
1,2,120,5,10,10
2,3,135,5,3,9
3,4,90,4,6,3
4,5,125,8,2,8
5,6,110,5,2,10
6,7,105,3,2,7


In [173]:
df_p1t2 = pd.read_excel('06_quiz_data.xlsx', sheet_name='p1_t2')
df_p1t2.columns = df_p1t2.columns.str.lower()
df_p1t2

,material,supply limit
0,1,100
1,2,150
2,3,200


Formulate a linear integer program that generates a feasible production plan to maximize the total profit (which is also the total revenue, as there is no cost in this problem). Then write a computer program (e.g., using MS Excel solver) to solve this instance and obtain an optimal plan. Do not set the production quantities to be integer; leave them fractional. After you find an optimal solution and its objective value, write down the integer part of the objective value as your solution (i.e., rounding down that value to the closest integer).

### Declarando o model

instanciando o model https://pyomo.readthedocs.io/en/stable/

In [174]:
model = pyo.ConcreteModel()

extraindo lista de materiais disponíveis

In [175]:
list_product = df_p1t1['produtos'].unique()
list_material = df_p1t2['material'].unique()

### Set

declarando set (index) do model

In [176]:
model.product = pyo.Set(initialize=list_product, doc='Lista de produtos disponíveis')
model.material = pyo.Set(initialize=list_material, doc='Lista de materiais disponíveis')

### Parâmetros

#### Material

In [177]:
dict_max_volume = df_p1t2.set_index('material')['supply limit'].to_dict()
dict_max_volume

{1: 100, 2: 150, 3: 200}

**declarando parâmetros**

In [178]:
model.supply_material = pyo.Param(model.material, initialize=dict_max_volume, doc='capacidade máxima de cada materal')
model.supply_material.display()

supply_material : capacidade máxima de cada materal
    Size=3, Index=material, Domain=Any, Default=None, Mutable=False
    Key : Value
      1 :   100
      2 :   150
      3 :   200


#### Produtos

In [179]:
new_itens = df_p1t1.set_index(['produtos'])
dict_preco = new_itens['preco'].to_dict()
dict_material_1 = new_itens['material_1'].to_dict()
dict_material_2 = new_itens['material_2'].to_dict()
dict_material_3 = new_itens['material_3'].to_dict()

#### declarando parâmetros 

In [181]:
model.preco = pyo.Param(model.product, initialize=dict_preco, doc='preco de cada produto ser escolhido')
model.material_1 = pyo.Param(model.product, initialize=dict_material_1, doc='quantidade de material 1 que o produto requer')
model.material_2 = pyo.Param(model.product, initialize=dict_material_2, doc='quantidade de material 2 que o produto requer')
model.material_3 = pyo.Param(model.product, initialize=dict_material_3, doc='quantidade de material 3 que o produto requer')

In [182]:
model.material_1.display()

material_1 : quantidade de material 1 que o produto requer
    Size=7, Index=product, Domain=Any, Default=None, Mutable=False
    Key : Value
      1 :     0
      2 :     5
      3 :     5
      4 :     4
      5 :     8
      6 :     5
      7 :     3


In [183]:
model.material_2.display()

material_2 : quantidade de material 2 que o produto requer
    Size=7, Index=product, Domain=Any, Default=None, Mutable=False
    Key : Value
      1 :     3
      2 :    10
      3 :     3
      4 :     6
      5 :     2
      6 :     2
      7 :     2


In [184]:
model.material_3.display()

material_3 : quantidade de material 3 que o produto requer
    Size=7, Index=product, Domain=Any, Default=None, Mutable=False
    Key : Value
      1 :    10
      2 :    10
      3 :     9
      4 :     3
      5 :     8
      6 :    10
      7 :     7


### Variáveis

decide se o item vai entrar na backpack

In [186]:
model.QTD_PRODUCT = pyo.Var(
    model.product,
    within=pyo.Reals,
    bounds=(0, None),
    doc='Variavel que indica a quantidade de produto produzido'
)

In [187]:
model.QTD_PRODUCT.display()

QTD_PRODUCT : Variavel que indica a quantidade de produto produzido
    Size=7, Index=product
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :  None :  None : False :  True :  Reals
      2 :     0 :  None :  None : False :  True :  Reals
      3 :     0 :  None :  None : False :  True :  Reals
      4 :     0 :  None :  None : False :  True :  Reals
      5 :     0 :  None :  None : False :  True :  Reals
      6 :     0 :  None :  None : False :  True :  Reals
      7 :     0 :  None :  None : False :  True :  Reals


### Restrições

respeitar a capacidade de material

In [188]:
model.del_component('limite_material_product')

In [189]:
def limit_capacity_material(model, material):
    if material == 1:
        return (
            sum(model.QTD_PRODUCT[product] * model.material_1[product] for product in model.product) <= model.supply_material[material]
        )
    elif material == 2:
        return (
            sum(model.QTD_PRODUCT[product] * model.material_2[product] for product in model.product) <= model.supply_material[material]
        )
    else:
        return (
            sum(model.QTD_PRODUCT[product] * model.material_3[product] for product in model.product) <= model.supply_material[material]
        )

model.limite_material_product = pyo.Constraint(model.material, rule=limit_capacity_material)

### Função Objetivo

máximiza value dos produtos produzidos

In [190]:
def maximiza_value(model):
    return sum(model.QTD_PRODUCT[product] * model.preco[product] for product in model.product)

model.objetivo = pyo.Objective(rule=maximiza_value, sense=pyo.maximize)

In [191]:
model.objetivo.pprint()

objetivo : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : maximize : 100*QTD_PRODUCT[1] + 120*QTD_PRODUCT[2] + 135*QTD_PRODUCT[3] + 90*QTD_PRODUCT[4] + 125*QTD_PRODUCT[5] + 110*QTD_PRODUCT[6] + 105*QTD_PRODUCT[7]


## Executando o model

### Escrevendo o model em arquivo lp

In [216]:
model.write(os.path.join('models', 'problema_1.lp'), io_options={'symbolic_solver_labels': True})

('models/problema_1.lp', 140593841696384)

### Definindo solver

In [193]:
opt = pyo.SolverFactory('glpk')

### Resolvendo o model

In [194]:
results = opt.solve(model)
results.solver.termination_condition

<TerminationCondition.optimal: 'optimal'>

In [212]:
model.objetivo.display()

objetivo : Size=1, Index=None, Active=True
    Key  : Active : Value
    None :   True : 3404.4585987261144


## Obtendo solução do problema

### Extraindo dados para um dataframe

In [213]:
df_final = pd.DataFrame(columns = ['produtos', 'preco', 'material_1', 'material_2', 'material_3', 'qtd_item'])

for product in model.product:

    qtd_item = model.QTD_PRODUCT[product].value
    preco = model.preco[product]
    material_1 = model.material_1[product]
    material_2 = model.material_2[product]
    material_3 = model.material_3[product]

    row = {
        'produtos':product,
        'preco':preco,
        'material_1':material_1,
        'material_2':material_2,
        'material_3':material_3,
        'qtd_item':qtd_item
    }
    
    
    df_intermediaro = pd.DataFrame([row])
    
    df_final = pd.concat([df_final, df_intermediaro], ignore_index=True)

df_final

,produtos,preco,material_1,material_2,material_3,qtd_item
0,1,100,0,3,10,7.961783
1,2,120,5,10,10,0.0
2,3,135,5,3,9,0.0
3,4,90,4,6,3,17.834395
4,5,125,8,2,8,0.0
5,6,110,5,2,10,0.0
6,7,105,3,2,7,9.55414


In [214]:
(
    df_final
    .query('qtd_item > 0')
    .assign(mult = lambda x: x['preco'] * x['qtd_item'])
    ['mult'].sum()
)

3404.4585987261144

# Problema 2

Consider a set of data $(x_{i}, y_{i}), i = 1, ..., n$, provided in Table 3. If we believe that $x_{i}$ and $y_{i}$​
has a linear relationship, we may apply simple linear regression to fit these data to a linear model. More precisely, we try to find $\alpha$ and $\beta$ such that the straight line $y = \alpha + \beta*x$ minimizes the sum of squared errors for all the data points:

$$
\underset{\alpha, \beta}{min} \sum_{i=1}^{n}\Bigl[ y_{i} - (\alpha + \beta x_{i})\Bigr]^2
$$

In [3]:
df_p2t3 = pd.read_excel('06_quiz_data.xlsx', sheet_name='p2_t3')
df_p2t3

,x,y
0,38,137
1,56,201
2,50,152
3,52,107
4,37,150
5,60,173
6,67,194
7,54,166
8,59,154
9,43,137


While almost all statistical software and packages provide tools for one to solve the above linear regression problem, we may also consider it as a nonlinear program and solve it with an optimization solver (e.g., MS Excel solver). For the data provided in the following table, solve the linear regression problem. Write down the optimal \betaβ you find by rounding it to the first digit after the decimal point (e.g., 9.011, 1.229, and 3.245 should be rounded to 9.01, 1.23, and 3.25 to be written down).

In [4]:
model = pyo.ConcreteModel()

In [5]:
model.x = df_p2t3['x'].unique()
model.y = df_p2t3['y'].unique()

In [6]:
model.a = pyo.Var(within=pyo.Reals)
model.b = pyo.Var(within=pyo.Reals)

In [7]:
def minimize_value(model):
    return sum((y - (model.a + model.b*x))**2 for x, y in zip(model.x, model.y))

model.objetivo = pyo.Objective(rule=minimize_value, sense=pyo.minimize)

In [8]:
model.objetivo.pprint()

objetivo : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : (137 - (a + 38*b))**2 + (201 - (a + 56*b))**2 + (152 - (a + 50*b))**2 + (107 - (a + 52*b))**2 + (150 - (a + 37*b))**2 + (173 - (a + 60*b))**2 + (194 - (a + 67*b))**2 + (166 - (a + 54*b))**2 + (154 - (a + 59*b))**2 + (38 - (a + 43*b))**2 + (193 - (a + 30*b))**2 + (175 - (a + 53*b))**2 + (247 - (a + 40*b))**2


In [9]:
model.write(os.path.join('models', 'problema_2.lp'), io_options={'symbolic_solver_labels': True})

('models/problema_2.lp', 140658600536720)

In [12]:
opt.options['tol'] = 5
opt.options['print_level'] = 12

opt = pyo.SolverFactory('ipopt')

In [14]:
results = opt.solve(model)
results.solver.termination_condition

ERROR: Solver (ipopt) returned non-zero return code (127)
ERROR: Solver log: /usr/local/bin/ipopt: error while loading shared libraries:
    libipopt.so.3: cannot open shared object file: No such file or directory


ApplicationError: Solver (ipopt) did not exit normally

In [ ]:
model.objetivo.display()